In [ ]:
#For Kaggle
#date=''

#For WL data
date='20241118'

saveImg=False

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
reload(fd)
reload(sd)

print('done')

In [ ]:
import frauddetection as fd
import pandas as pd 
   
dfTrx = fd.read_file('../data/raw/'+sd.source+'export'+date+'.csv')
dfTrx.head(5)

# Remove unless columns

In [ ]:
# remove db_uuid issuer_id cluster_profile

dfTrx2= dfTrx.drop(columns=['db_uuid','issuer_id', 'cluster_profile'])
dfTrx2.head()

# Fill missing values

In [ ]:
import numpy as np

nan_count = dfTrx2.isnull().sum()
print('Number of NaN values:', nan_count)


dfTrx2['mcd_fraud_score'].fillna(np.mean(dfTrx2['mcd_fraud_score']), inplace=True)
dfTrx2['vaa_score'].fillna(np.mean(dfTrx['vaa_score']), inplace=True)
dfTrx2.head()